## Check List 3.2 - Combined Detector (ArucoStero, MultiICP)  

* This test is for checking combined detection functionality with Aruco and MultiICP. Two cameras and one cup is used as shown below. 


* **3.2.1 Combined Detector**  
  - initialize, set_config, detect_and_register, disconnect   

* **Test setup**
  - Camera: Kinect (Main), Realsense(Sub)
  - Target: Cup
<img src="./Figs/3.2.CupDetection.jpg" width="80%">
  
* **TBD**
  - Auto initialization to estimate initial guess for ICP is not perfect
  - Robust and reliable initial guess for gloabl registration will be done
  - Multiple instance for the same class will be done

## Set running directory to Project source

In [1]:
import os
import sys
import numpy as np
import cv2
import copy
import matplotlib.pyplot as plt
sys.path.append(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

In [2]:
from pkg.global_config import RNB_PLANNING_DIR
from pkg.utils.utils import *    
from pkg.utils.rotation_utils import *
from pkg.controller.combined_robot import *
from pkg.geometry.builder.scene_builder import SceneBuilder
from pkg.geometry.geometry import GeometryItem
from pkg.geometry.geotype import GEOTYPE
from pkg.detector.detector_interface import DetectionLevel
from pkg.detector.multiICP.config import *

## 3.2.1 Combined Detector

In [3]:
from pkg.detector.combined_detector import CombinedDetector
from pkg.detector.multiICP.multiICP import MultiICP, MultiICP_Obj
from pkg.detector.aruco.stereo import ArucoStereo
from pkg.detector.aruco.marker_config import get_aruco_map
from pkg.detector.camera.kinect import Kinect
from pkg.detector.camera.realsense import RealSense
from pkg.detector.detector_interface import DetectionLevel

##### create cameras

In [4]:
kn = Kinect()
rs = RealSense()

##### create ArucoStereo instance

In [5]:
stereo = ArucoStereo(aruco_map=get_aruco_map(), 
                     camera_list=[kn, rs])
stereo.initialize()

time.sleep(1) # Let the camera have some time to get stable
stereo.calibrate()

Device configuration: 
	color_format: 3 
	(0:JPG, 1:NV12, 2:YUY2, 3:BGRA32)

	color_resolution: 5 
	(0:OFF, 1:720p, 2:1080p, 3:1440p, 4:1536p, 5:2160p, 6:3072p)

	depth_mode: 3 
	(0:OFF, 1:NFOV_2X2BINNED, 2:NFOV_UNBINNED,3:WFOV_2X2BINNED, 4:WFOV_UNBINNED, 5:Passive IR)

	camera_fps: 2 
	(0:5 FPS, 1:15 FPS, 2:30 FPS)

	synchronized_images_only: False 
	(True of False). Drop images if the color and depth are not synchronized

	depth_delay_off_color_usec: 0 ms. 
	Delay between the color image and the depth image

	wired_sync_mode: 0
	(0:Standalone mode, 1:Master mode, 2:Subordinate mode)

	subordinate_delay_off_master_usec: 0 ms.
	The external synchronization timing.

	disable_streaming_indicator: False 
	(True or False). Streaming indicator automatically turns on when the color or depth camera's are in use.


Start streaming


[(array([[1.82983423e+03, 0.00000000e+00, 1.91572046e+03],
         [0.00000000e+00, 1.82983423e+03, 1.09876086e+03],
         [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
  array([ 7.09966481e-01, -2.73409390e+00,  1.45804870e-03, -3.24774766e-04,
          1.44911301e+00,  5.84310412e-01, -2.56374550e+00,  1.38472950e+00]),
  0.001),
 (array([[1.39560388e+03, 0.00000000e+00, 9.62751587e+02],
         [0.00000000e+00, 1.39531934e+03, 5.47687012e+02],
         [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
  array([0., 0., 0., 0., 0.]),
  0.0010000000474974513),
 array([[ 0.81117624, -0.03337384, -0.5838487 ,  0.7141189 ],
        [ 0.14473428,  0.9787678 ,  0.14513993, -0.14857051],
        [ 0.5666084 , -0.202237  ,  0.7987835 ,  0.05353302],
        [ 0.        ,  0.        ,  0.        ,  1.        ]],
       dtype=float32)]

##### create multiICP instance

In [6]:
micp = MultiICP(kn)
micp.initialize()

Device configuration: 
	color_format: 3 
	(0:JPG, 1:NV12, 2:YUY2, 3:BGRA32)

	color_resolution: 5 
	(0:OFF, 1:720p, 2:1080p, 3:1440p, 4:1536p, 5:2160p, 6:3072p)

	depth_mode: 3 
	(0:OFF, 1:NFOV_2X2BINNED, 2:NFOV_UNBINNED,3:WFOV_2X2BINNED, 4:WFOV_UNBINNED, 5:Passive IR)

	camera_fps: 2 
	(0:5 FPS, 1:15 FPS, 2:30 FPS)

	synchronized_images_only: False 
	(True of False). Drop images if the color and depth are not synchronized

	depth_delay_off_color_usec: 0 ms. 
	Delay between the color image and the depth image

	wired_sync_mode: 0
	(0:Standalone mode, 1:Master mode, 2:Subordinate mode)

	subordinate_delay_off_master_usec: 0 ms.
	The external synchronization timing.

	disable_streaming_indicator: False 
	(True or False). Streaming indicator automatically turns on when the color or depth camera's are in use.


Initialize Done


In [7]:
detector = CombinedDetector([stereo, micp])

##### create SceneBuilder instance

In [8]:
scene_builder = SceneBuilder.instance(detector=detector)

##### set reference coordinate and viewpoint (by StereoAruco)

In [9]:
T_kn = stereo.ref_coord_inv

##### create geometry scene

In [10]:
crob = CombinedRobot(
    robots_on_scene=[
        RobotConfig(0, RobotType.indy7, ((0,-0.3,0), (0,0,0)), None),
        RobotConfig(1, RobotType.panda, ((0,0.3,0), (0,0,0)), None)], 
    connection_list=[False, False])

xyz_rpy_robots = scene_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = scene_builder.create_gscene(crob)
gscene.show_pose(crob.home_pose)
viewpoint = gscene.create_safe(gtype=GEOTYPE.SPHERE, name="viewpoint", link_name="base_link",
                               dims=(0.05, 0.05, 0.02), center=T_kn[:3,3], rpy=Rot2rpy(T_kn[:3,:3]),
                               color=(1, 0, 0, 0.3), display=True, fixed=True, collision=False)
viewpoint.draw_traj_coords([crob.home_pose])

connection command:
indy0: False
panda1: False
name_mask is []
[WARN] CombinedRobot is not set: call set_config()
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker


[array([[ 0.99967021, -0.01577677, -0.02026191,  0.02112267],
        [ 0.00505459, -0.65269959,  0.75759995, -0.51821285],
        [-0.02517742, -0.75745255, -0.65240461,  0.5546183 ],
        [ 0.        ,  0.        ,  0.        ,  1.        ]])]

### Set MultiICP configs
* You have to make micp, hrule and grule for each object you want to detect

* hrule means heuristic rule for special object which cannot be detected thorugh mask rcnn using COCO dataset

* grule means initial guess(R,t) for ICP

* Run shraed detector to detect object in color image

##### (FYI) Running shared detector for object detection on bash - only when you want to keep detector server for multiple program runs
```bash
python3 /home/jhkim/Projects/rnb-planning/src/pkg/detector/multiICP/shared_detector.py --dims='(720,1280,3)'
```

#### Clearing shared detector channels if zombie memory remains
```python
from pkg.utils.shared_function import clear_channels_on, sa
clear_channels_on("SharedDetector")
```

In [11]:
from pkg.utils.shared_function import clear_channels_on, sa
clear_channels_on("SharedDetector")

In [12]:
from pkg.detector.multiICP.shared_detector import SharedDetectorGen
sd = SharedDetectorGen(tuple(reversed(micp.dsize))+(3,))()
sd.init()

[WARN] Data size too big for shared function: 165888000 - shared_fun.inference.shareddetector.0


In [13]:
obj_info_dict = get_obj_info()

In [14]:
# object items you want to detect
# heuristic rule items, Initial guess rule items
micp_cup = MultiICP_Obj(obj_info_dict["cup"], None,
                        OffsetOnModelCoord("cup", R=Rot_axis(1, np.pi/2), offset=None))

In [15]:
micp_dict = {"cup": micp_cup}
micp.set_config(micp_dict, sd, crob, viewpoint)

##### detect_and_register()
* Detect items in the field of view and register them to the GeometryScene
* They will appear in the RVIZ



In [16]:
gtem_dict = scene_builder.detect_and_register(level_mask=[DetectionLevel.ENVIRONMENT])

name_mask is []


In [20]:
gtem_dict = scene_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE], 
                                              visualize=True)

name_mask is ['cup']
===== Detected : cup, 1 object(s) =====

'cup' is not in gscene. Use manual input for initial guess

Apply point-to-point ICP
registration::RegistrationResult with fitness=1.000000e+00, inlier_rmse=1.535097e-02, and correspondence_set size of 3881
Access transformation to get result.
Transformation is:
[[ 0.61290073 -0.78971375 -0.02654972 -0.02437707]
 [-0.39108137 -0.27398116 -0.87862944 -0.08261382]
 [ 0.68659163  0.54889572 -0.47676559  0.64561296]
 [ 0.          0.          0.          1.        ]]
initial: 
[[ 0.61 -0.79 -0.03 -0.02]
 [-0.39 -0.27 -0.88 -0.08]
 [ 0.69  0.55 -0.48  0.65]
 [ 0.    0.    0.    1.  ]]
Apply point-to-point ICP
registration::RegistrationResult with fitness=1.000000e+00, inlier_rmse=5.653576e-03, and correspondence_set size of 1801
Access transformation to get result.
Transformation is:
[[ 0.48073611 -0.86623679 -0.13611254 -0.01041573]
 [-0.50923985 -0.14943675 -0.84755144 -0.07405812]
 [ 0.71384002  0.47676251 -0.51296193  0.66897

##### disconnect()

In [21]:
detector.disconnect()